In [46]:
import pandas as pd
import numpy as np
import glob
import os
from lstm_model import *
from sklearn.linear_model import Ridge

In [2]:
files = ['all_rocket.csv','all_attlstm.csv','all_resnet.csv','IF.csv']
df = pd.read_csv(files[0])
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df[df.columns[0]]))) 
df2 = pd.read_csv(files[1])
df2 = df2.set_index(pd.DatetimeIndex(pd.to_datetime(df2[df2.columns[0]]))) 
df3 = pd.read_csv(files[2])
df3 = df3.set_index(pd.DatetimeIndex(pd.to_datetime(df3[df3.columns[0]])))
df4 = pd.read_csv(files[3])
df4 = df4.set_index(pd.DatetimeIndex(pd.to_datetime(df4[df4.columns[0]])))

In [3]:
data = df.join(df2,lsuffix='att').join(df3,lsuffix='res').join(df4,lsuffix='lin')

In [4]:
data = data.dropna()
data = data[['IFatt','IFres','IFlin','IF']]

In [5]:
target = pd.read_csv('y.csv')

In [6]:
target = target.set_index(pd.DatetimeIndex(pd.to_datetime(target[target.columns[0]])))
data = data.join(pd.DataFrame(target.y_10m))

In [7]:
target = data.y_10m
data = data.drop(['y_10m'],axis=1)
index = data.index

In [48]:
data = np.array(data)#.reshape(len(data),data.shape[2])

In [9]:
target = np.array(target).reshape(len(target),)

In [10]:
import xgboost as xgb

In [73]:
classifier = Ridge()
parameters = {
                'alpha': [25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01,0.001,0.0001,0]
                }
clf = GridSearchCV(classifier,
                           parameters,
                           verbose=1, 
                           scoring="neg_mean_squared_error",
                           cv=5,
                           n_jobs=-1,
                  )
clf.fit(data[:int(1e6),:3],target[:int(1e6)])

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    9.0s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'alpha': [25, 10, 4, 2, 1.0, 0.8, 0.5, 0.3, 0.2, 0.1, 0.05, 0.02, 0.01, 0.001, 0.0001, 0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=1)

In [96]:
classifier = Ridge(alpha=3, fit_intercept=False)
classifier.fit(data[:int(1e6),:3],target[:int(1e6)])

Ridge(alpha=3, copy_X=True, fit_intercept=False, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [97]:
y_pred = classifier.predict(data[int(1e6):,:3])
#y_pred = np.concatenate(y_pred)
#print("accuracy_score" + str(accuracy_score(np.sign(np.array(target_test)),np.sign(y_pred))))
lin = np.linspace(0,1,5)
print(pd.DataFrame(y_pred).quantile(lin))

             0
0.00 -0.000244
0.25 -0.000036
0.50 -0.000020
0.75 -0.000002
1.00  0.000246


In [98]:
print("cor: " + str(np.corrcoef(target[int(1e6):],y_pred)[0,1]))
pd.DataFrame({'index':index[int(1e6):],'y_pred':y_pred}).to_csv("all.csv",index=False)

cor: 0.029512706307931146


In [99]:
coefs = classifier.coef_/sum(classifier.coef_)
coefs

array([ 0.47114044, -0.04624457,  0.57510412])